## Preparing for pre-processing

### Installing and importing required modules

In [ ]:
%pip install nltk

In [10]:
import sqlite3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

### Connecting to the database and getting posts

In [12]:
conn = sqlite3.connect('/home/sh1ron/HSE/MLOps_pipeline/database.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM posts")
posts = cursor.fetchall()

conn.close()

## Data preprocessing

### Post tokenisation

In [13]:
tokenized_posts = []
for post in posts:
   tokenized_post = word_tokenize(post[2])
   tokenized_posts.append(tokenized_post)

### Removing stopwords

In [14]:
stop_words = set(stopwords.words('russian'))

filtered_posts = []
for post in tokenized_posts:
  filtered_post = [word for word in post if word not in stop_words]
  filtered_posts.append(filtered_post)

['🎅', 'Кто-то', 'году', 'ооочень', 'вёл']
